In [1]:
#import pyspark
#sc = pyspark.SparkContext.getOrCreate()
#sqlContext = pyspark.sql.SQLContext(sc)
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType
import pandas as pd
import xlrd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
class project:
    def read_excel_xlrd(path):
        xl_workbook = xlrd.open_workbook(path)
        #sheet_names = xl_workbook.sheet_names()
        #print('Sheet Names', sheet_names)
        #xl_sheet = xl_workbook.sheet_by_name(sheet_names[0])
        xl_sheet = xl_workbook.sheet_by_index(0)
        #print ('Sheet name: %s' % xl_sheet.name)
        data = []
        for row in range(xl_sheet.nrows):
            line = []
            for col in range(xl_sheet.ncols):
                line.append(str(xl_sheet.cell(row, col).value))
            data.append(line)
        columnNames = data[0]
        dataValues = data[1:]
        dataPairs = []
        for row in range(len(dataValues)):
            line = {}
            for col in range(len(columnNames)):
                line[columnNames[col]] = dataValues[row][col]
            dataPairs.append(Row(**line))
        dataRDD = sc.parallelize(dataPairs)
        df = dataRDD.toDF()
        return df
    
    def topics_get(df, columnName, delimiter):
        rows = df.groupBy(columnName).count().collect()
        results = []
        for row in rows:
            for item in row[columnName].split(delimiter):
                if item not in results:
                    results.append(item)
        results.sort()
        return results
    
    def exist_list(list_all, list_compared):
        results = []
        for item in list_all:
            if item in list_compared:
                results.append(1)
            else:
                results.append(0)
        return results
    
    def df_add_withColumn(df, columnName_input, columnName_output, function, returnType):
        function_udf = udf(lambda item: function, returnType)
        return df.withColumn(columnName_output, function_udf(col(columnName_input)))
    

In [4]:
df = project.read_excel_xlrd("lib-statistics.xlsx")
print(df.filter(df['Ödünç Sayısı'].isNull()).count())
print(df.filter(df['Ödünç Sayısı']=='').count())
df = df.withColumn('Sınıflama', split(df['Sınıflama'], " ")[0])
df = df.withColumn('Ödünç Sayısı', df['Ödünç Sayısı'].cast('float'))
print(df.filter(df['Ödünç Sayısı'].isNull()).count())
df = df.fillna({'Sınıflama' : 'noInfo', 'Eser Adı' : 'noInfo', 'Yazar' : 'noInfo', 'Dil' : 'noInfo', 'Konu Başlıkları' : 'noInfo' , 'Ödünç Sayısı' : 0.0})
print(df.filter(df['Ödünç Sayısı'].isNull()).count())

0
598
598
0


In [5]:
df.select(sum(df['Ödünç Sayısı'])).show()

+-----------------+
|sum(Ödünç Sayısı)|
+-----------------+
|            339.0|
+-----------------+



In [6]:
df.printSchema()

root
 |-- Dil: string (nullable = false)
 |-- Eser Adı: string (nullable = false)
 |-- Konu Başlıkları: string (nullable = false)
 |-- Sınıflama: string (nullable = false)
 |-- Yazar: string (nullable = false)
 |-- Ödünç Sayısı: float (nullable = false)



In [7]:
df.show()

+---+--------------------+--------------------+---------+--------------------+------------+
|Dil|            Eser Adı|     Konu Başlıkları|Sınıflama|               Yazar|Ödünç Sayısı|
+---+--------------------+--------------------+---------+--------------------+------------+
|eng|Museum frictions ...|Museums--Social A...|       AM|                    |         0.0|
|tur|Tahayyül gücünü y...|Imagination (Phil...|        B|                    |         0.0|
|tur|Bir yol var : Min...|Ontology
Stress M...|        B|        Damcı, Taner|         0.0|
|tur|Psikoloji şerhi =...|Aristotle
Aristot...|        B|            İbn Rüşd|         2.0|
|eng|An examination of...|Özel, İsmet, 1944...|        B|      Kaya, Vefa Can|         0.0|
|eng|An examination of...|Özel, İsmet, 1944...|        B|      Kaya, Vefa Can|         0.0|
|tur|Ruhun uyanışı, ya...|Philosophy, Islam...|        B|İbn Ṭufeyl, Muḥam...|         0.0|
|tur|Çağımızın sorunla...|Philosophy, Moder...|        B|   Russell, Bertrand|  

In [8]:
df.collect()[0]['Konu Başlıkları']

'Museums--Social Aspects--Congresses\nMuseums--Economic Aspects--Congresses\nMuseums--Political Aspects--Congresses\nGlobalization--Social Aspects--Congresses\nMuseum Techniques--Congresses\nMuseum Exhibits--Congresses\nPopular Culture--Congresses\nCulture Diffusion--Congresses\nCultural Pluralism--Congresses\nMüzeler--Sosyal Yönler--Kongreler\nMüzeler--Ekonomik Yönler--Kongreler\nMüzeler--Siyasi Yönler--Kongreler\nKüreselleşme--Sosyal Yönler--Kongreler\nMüze Teknikleri--Kongreler\nMüze Sergileri--Kongreler\nPopüler Kültür--Kongreler\nKültür Yayılımı--Kongreler\nKültürel Çoğulculuk--Kongreler'

In [9]:
df.filter(df['Konu Başlıkları']=='').collect()

[Row(Dil='tur', Eser Adı='Edebiyat tarihi yazıları : arşiv kayıtları, yazma eserler ve kayıp metinler', Konu Başlıkları='', Sınıflama='PL', Yazar='Erünsal, İsmail E.', Ödünç Sayısı=1.0)]

In [10]:
df.filter(df['Konu Başlıkları']=='noInfo').collect()

[]

In [11]:
df.filter(df['Konu Başlıkları']=='None').collect()

[]

In [12]:
df.filter(df['Konu Başlıkları'].isNull()).collect()

[]

In [13]:
def countEmptyAndNull(df):
    for columnName in df.columns:
        print(r"Column '%s' has %d '', %d 'noInfo', %d 'None', %d filled, and %d null rows." 
              % (columnName, df.filter(df[columnName]=='').count(), df.filter(df[columnName]=='noInfo').count(),
                 df.filter(df[columnName]=='None').count(), df.filter(df[columnName]!='').count(),
                 df.filter(df[columnName].isNull()).count()))


In [14]:
countEmptyAndNull(df)

Column 'Dil' has 0 '', 0 'noInfo', 0 'None', 751 filled, and 0 null rows.
Column 'Eser Adı' has 0 '', 0 'noInfo', 0 'None', 751 filled, and 0 null rows.
Column 'Konu Başlıkları' has 1 '', 0 'noInfo', 0 'None', 750 filled, and 0 null rows.
Column 'Sınıflama' has 0 '', 0 'noInfo', 0 'None', 751 filled, and 0 null rows.
Column 'Yazar' has 294 '', 0 'noInfo', 0 'None', 457 filled, and 0 null rows.
Column 'Ödünç Sayısı' has 0 '', 0 'noInfo', 0 'None', 0 filled, and 0 null rows.


In [15]:
df.count()

751

In [16]:
topics = project.topics_get(df, 'Konu Başlıkları', '\n')
print(topics)

['', 'Abraham, Ecchellensis, 1605-1664--Congresses', 'Abraham, Ecchellensis, 1605-1664--Kongreler', 'Acara (Gürcistan)--Görsel Çalışmalar', 'Action Films', 'Adalet', 'Administrative Jurisdiction--Turkey', 'Administrative Law--Turkey', 'Adventure Films', 'Advices', 'Advisory Council--Turkey--History--Periodicals', 'Advisory Council--Turkey--Proceedings--Periodicals', 'Aegean Islands (Greece and Turkey)', 'Aegean Islands (Greece and Turkey)--Politics and Government', 'Aegean Sea--History', 'Africa, North--Relations--Italy--Congresses', 'African Americans--Civil Rights--History--20th Century--Sources', 'African Literature--History and Criticism', 'Afrika Edebiyatı--Tarih ve Eleştiri', 'Afrika, Kuzey--İlişkiler--İtalya--Kongreler', 'Afrikalı Amerikalılar--Medeni Haklar--Tarih--20. Yüzyıl--Kaynaklar', 'Agricultural Laborers--Great Britain--History--19th Century', 'Agriculture--Turkey--Statistics', 'Ahlaki Eğitim--Süreli Yayınlar', 'Ahmed Şefik Midhat Paşa, 1822-1884', 'Ahmet Paşa, Kimbarac

In [17]:
#df2 = df.groupBy('Dil').count()
#df2.select(sum(df2['count'])).show()
#df.select(sum(df['Ödünç Sayısı'])).show()
#df.filter(df['Ödünç Sayısı'].isNull()).count()
#df.groupBy(df['Sınıflama']).count().show()
#df_s = df.groupBy(df['Sınıflama']).count()
#print(df_s.filter(df_s['count'] > 1).count())
#print(df_s.filter(df_s['count'] < 1).count())
#print(df_s.filter(df_s['count'] == 1).count())

In [18]:
#df_s.filter(df_s['Sınıflama'].like('%D%')).show()
#df_s.filter(split(df_s['Sınıflama'], " ")[0].like("%D%")).show()
#df_s.groupBy(split(df_s['Sınıflama'], " ")[0]).count().show()
#df_s.groupBy(split(df_s['Sınıflama'], " ")[0].alias("group")).count().orderBy("group").collect()

In [19]:
#df.groupBy(df['Sınıflama']).agg(collect_list('Konu Başlıkları').alias('gruplanmış konu başlıkları')).show()
#df.groupBy(df['Sınıflama']).agg(collect_list('Konu Başlıkları'), collect_list('Yazar')).show()
#df.filter(df['Ödünç Sayısı'] != 0.0).select("Eser Adı").collect()

In [20]:
#df3 = df.groupBy(split(split(df['Sınıflama'], " ")[0], "")[0].alias("group")).count().orderBy("group")
#group_count = df3.count()
#groups = np.empty(group_count, dtype="S30")
#counts = np.empty(group_count)
#for (index,row) in enumerate(df3.collect()):
#    groups[index] = row['group']
#    counts[index] = row['count']
#sns.barplot(x=groups, y=counts)
#plt.xticks(rotation='vertical')
#sns.despine()

In [21]:
#df4 = df.groupBy(df['Dil']).count().orderBy('Dil')
#lang_count = df4.count()
#langs = np.empty(lang_count, dtype="S30")
#counts = np.empty(lang_count)
#for (index,row) in enumerate(df4.collect()):
#    langs[index] = row['Dil']
#    counts[index] = row['count']
#sns.barplot(x=langs, y=counts)
#plt.xticks(rotation='vertical')
#sns.despine()

In [22]:
# Imports
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
# First we create a RDD in order to create a dataFrame:
rdd = sc.parallelize([('u1', 1, [1 ,2, 3]), ('u1', 4, [1, 2, 3])])
df2 = rdd.toDF(['user', 'item', 'fav_items'])
# Print dataFrame
df2.show()

# We make an user define function that receives two columns and do operation
function = udf(lambda item, items: 1 if item in items else 0, IntegerType())

df2.select('user', 'item', 'fav_items', function(col('item'), col('fav_items')).alias('result')).show()

+----+----+---------+
|user|item|fav_items|
+----+----+---------+
|  u1|   1|[1, 2, 3]|
|  u1|   4|[1, 2, 3]|
+----+----+---------+

+----+----+---------+------+
|user|item|fav_items|result|
+----+----+---------+------+
|  u1|   1|[1, 2, 3]|     1|
|  u1|   4|[1, 2, 3]|     0|
+----+----+---------+------+



In [23]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType

list_all1 = ['a', 'b', 'c', 'd', 'e']

rdd = sc.parallelize([('u1', 'a\nb'), ('u1', 'd\ne\na')])
df1 = rdd.toDF(['user', 'items'])

df1.show()

def exist_list(list_all, list_compared):
    results = []
    for item in list_all:
        if item in list_compared:
            results.append(1)
        else:
            results.append(0)
    return results

function = udf(lambda item: exist_list(list_all1, item.split('\n')), ArrayType(IntegerType()))

df1.withColumn('result', function(col('items'))).show()

+----+-----+
|user|items|
+----+-----+
|  u1|  a
b|
|  u1|d
e
a|
+----+-----+

+----+-----+---------------+
|user|items|         result|
+----+-----+---------------+
|  u1|  a
b|[1, 1, 0, 0, 0]|
|  u1|d
e
a|[1, 0, 0, 1, 1]|
+----+-----+---------------+



In [24]:
df.filter(df['Ödünç Sayısı'] != 0.0).select("Eser Adı").collect()

[Row(Eser Adı="Psikoloji şerhi =  Telhisu kitabi'n-nefs"),
 Row(Eser Adı='Çağımızın sorunları üstüne düşünceler'),
 Row(Eser Adı="Heidegger'in kulübesi"),
 Row(Eser Adı='Bana bilgiçlik taslayan adamlar = Men explain things to me'),
 Row(Eser Adı='Sahip olmak ya da olmak'),
 Row(Eser Adı='Geliştiren anne baba'),
 Row(Eser Adı='Research design and methods : a process approach'),
 Row(Eser Adı='Hayriyye-i Nabi : inceleme-metin'),
 Row(Eser Adı='Din psikolojisi'),
 Row(Eser Adı="Mitoloji Kur'an kıssaları ve tarihi gerçeklik"),
 Row(Eser Adı='Hadislerin dili : ilk hadis belgesi Hemmam’ın Sahifesi : tertip, terceme, yorum'),
 Row(Eser Adı='Güncel Dini Meseleler İstişare Toplantısı'),
 Row(Eser Adı='Neoliberal İslamcılık 1980-2015 : İslamcıların dünya sistemine entegrasyonu'),
 Row(Eser Adı='Cumhuriyet Türkiyesi’nde bir mesele olarak İslam'),
 Row(Eser Adı='Cumhuriyet Türkiyesi’nde bir mesele olarak İslam'),
 Row(Eser Adı='Kapitalizme eklemlenme : dindar orta sınıfta tüketim kültürü'),
 Row(E

In [25]:
df.filter(df['Ödünç Sayısı'] != 0.0).count()

153